In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/home/vice/DEZ/amazon_product_review/key/de-amazon-product-review-88dd24afade6.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/vice/DEZ/spark/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/04/25 15:53:49 WARN Utils: Your hostname, peace resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/25 15:53:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/25 15:53:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
df = spark.read.parquet('gs://de-amazon-product-review-bucket/staging/review/')


In [9]:
df.printSchema()

root
 |-- timestamp_ms: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- helpful_votes: long (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [10]:
df.show()

[Stage 1:>                                                          (0 + 1) / 1]

+-------------+------+-------------+--------------------+--------------------+----------+-----------------+-----------+--------------------+----+-----+---+-------------------+
| timestamp_ms|rating|helpful_votes|               title|                text|      asin|verified_purchase|parent_asin|             user_id|year|month|day|          timestamp|
+-------------+------+-------------+--------------------+--------------------+----------+-----------------+-----------+--------------------+----+-----+---+-------------------+
|1560778379315|   5.0|         null|     Would buy again|         Great value|B07FX1CS1P|             true| B07FX1CS1P|AEE35FJJU4ZC3PI7A...|2019|    6| 17|2019-06-17 21:32:59|
|1479920791000|   5.0|         null|Excellent throwba...|Excellent quality...|B00X08R0LQ|             true| B00X08R0LQ|AFQ7K4AQZ67TL5BWZ...|2016|   11| 23|2016-11-24 01:06:31|
|1355261513000|   5.0|         null|               yummy|this was a gift w...|B004GW7PI8|             true| B004GW7PI8|A

In [11]:
df.count()


571544897